In [4]:
!pip install influxdb-client


  Using cached pyspark-3.2.0.tar.gz (281.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
    Running setup.py install for pyspark: started
    Running setup.py install for pyspark: still running...
    Running setup.py install for pyspark: finished with status 'done'


In [7]:
!pip install pyspark

In [8]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

import os
import json

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'

In [ ]:
spark = SparkSession.builder.master('local[*]').getOrCreate()
spark.sparkContext.setLogLevel('ERROR')

In [ ]:
from pyspark.sql import *

class InfluxDBWriter:
    def __init__(self):
        cloud_url = "https://westeurope-1.azure.cloud2.influxdata.com"
        token = "iJHZR-dq4I5LIgFZCc5jTUNx-I7dyz29ZTO-B4W5DpU4mhPVDFg-aAb2jK4Vz1C6n0DDb6ddA-bJ3EZAanAOUw=="
        org = "ahr9oi@inf.elte.hu"
        self.bucket = "ahr9oi's Bucket"
        
        self.client = InfluxDBClient(url=cloud_url, token=token, org=org)
        self.write_api = self.client.write_api()

    def open(self, partition_id, epoch_id):
        print("Opened %d, %d" % (partition_id, epoch_id))
        return True

    def process(self, row):
        self._row_to_line_protocol(row)
        #self.write_api.write(bucket=self.bucket, record=self._row_to_line_protocol(row))

    def close(self, error):
        self.write_api.__del__()
        self.client.__del__()
        print("Closed with error: %s" % str(error))

    def _row_to_line_protocol(self, row: Row):
        # TODO map Row to LineProtocol
        row.show()